In [32]:
import pandas as pd
import os
import numpy as np

In [33]:
excel_table =  'funprof_geomosaic_kos.csv'
gthered_table = './counts/raw_counts_intersect_bp.tsv'

lista = [
"CR18_ER180415_F",
"CR18_ER180415_S",
"CR18_LE180416_F",
"CR18_LE180416_S",
"CR18_LW180405_F",
"CR18_LW180405_S",
"CR18_XF180416_F",
"CR18_XF180416_S"
]

In [34]:
table = pd.read_csv(excel_table, sep = ',')
print(table.head())

   Element        Relevance         cycle  Pathway  \
0  Arsenic  Biogeochemistry       Arsenic     AsOx   
1  Arsenic  Biogeochemistry       Arsenic     AsOx   
2  Arsenic  Biogeochemistry       Arsenic    AsRed   
3  Arsenic  Biogeochemistry       Arsenic    AsRed   
4   Carbon  Biogeochemistry  Fermentation  Acetate   

                                        pathway name  gene  \
0                                 Arsenite oxidation  aoxA   
1                                 Arsenite oxidation  aoxB   
2  Dissimilatory arsenic reduction (arsRBC or ars...  arsC   
3  Dissimilatory arsenic reduction (arsRBC or ars...  arsC   
4                                Mixed acid: acetate  poxB   

                                           gene_name key_gene energyRole  \
0  arsenite oxidase small subunit [EC:1.20.2.1 1....      yes          D   
1  arsenite oxidase large subunit [EC:1.20.2.1 1....      yes          D   
2                                 arsenate reductase      yes          A  

In [35]:
import math

def redox_index(acceptors_list: list, donors_list:list):
    
    num_donors = len(donors_list)
    num_acceptors = len(acceptors_list)
    
    # Compute the thoreatical maximum number of pairs 
    index = math.log(num_donors) + math.log(num_acceptors)
    # or equivalently    
    #index = math.log(num_donors * num_acceptors)
    
    return index



In [36]:
    
def get_KOS(working_dir:str,s:str):
    
    pckg = 'funprofiler'
    path_to_dir = os.path.join(working_dir,s,pckg)
    if os.path.exists(path_to_dir):
        table = os.path.join(path_to_dir,'prefetch_out.csv')
        results = pd.read_csv(table, sep = ',')

        ko_list = results["match_name"].str.split(':').str[1].unique().tolist()
        subset_ = results[["intersect_bp", "match_name"]]
        n_kos = len(ko_list)
    
    return n_kos,ko_list,subset_


In [37]:
def match_kos(unique_ko_list:list,spreadsheet:str):

    master_table = pd.read_csv(spreadsheet,sep = ',')
    # 1. Filter Sample KOs in our master table
    detected_mask = master_table['KO'].isin(unique_ko_list)
    filtered_df = master_table[detected_mask]
    # 2. Split into UNIQUE Donors (D) and Acceptors (A)
    donors_df = filtered_df[filtered_df['energyRole'] == 'D']
    acceptors_df = filtered_df[filtered_df['energyRole'] == 'A']
    # 3. Select cofactors
    donors = set(donors_df['KO'])
    acceptors = set(acceptors_df['KO'])
    
    metal_donors = donors_df['Metal'].dropna()
    metal_acceptrs = acceptors_df['Metal'].dropna()
    
    m_a = set(metal_acceptrs)
    m_d = set(metal_donors)

    return acceptors, donors, m_a, m_d

In [39]:
dictio = {}
all_A = []
all_D = []

raw_reads = pd.read_csv(gthered_table, sep = '\t')
for s in lista:

    sample_data = raw_reads[raw_reads[s] != 0][['ko_id', s]]
    kos = sample_data['ko_id'].unique()

    # #n,unique,raw_data = get_KOS(working_dir=working_dir,s=sample)
    print(f'Sample {s} has : {len(kos)} kos')
    acceptors, donors, m_a, m_d = match_kos(unique_ko_list=kos,spreadsheet=excel_table)
    all_D.append(donors)
    all_A.append(acceptors)
    
    print(f'Acceptors: {len(acceptors)}')
    print(f'Donors: {len(donors)}')
    index_ko_pairs = redox_index(acceptors,donors)
    index_metal_pairs = redox_index(m_a,m_d)
    print("RM index:", index_ko_pairs, '\n')
    print("RP index:", index_metal_pairs, '\n')
    
    dictio[s] = {'A': acceptors, 'D' : donors}

Sample CR18_ER180415_F has : 5676 kos
Acceptors: 52
Donors: 86
RM index: 8.405591014834934 

RP index: 4.852030263919617 

Sample CR18_ER180415_S has : 5158 kos
Acceptors: 49
Donors: 80
RM index: 8.273846932784508 

RP index: 4.584967478670571 

Sample CR18_LE180416_F has : 5949 kos
Acceptors: 57
Donors: 78
RM index: 8.399760094524142 

RP index: 4.653960350157523 

Sample CR18_LE180416_S has : 4793 kos
Acceptors: 46
Donors: 74
RM index: 8.132706489693266 

RP index: 4.584967478670571 

Sample CR18_LW180405_F has : 2241 kos
Acceptors: 36
Donors: 51
RM index: 7.515344571180435 

RP index: 4.787491742782046 

Sample CR18_LW180405_S has : 4591 kos
Acceptors: 53
Donors: 86
RM index: 8.42463920980563 

RP index: 4.969813299576001 

Sample CR18_XF180416_F has : 5748 kos
Acceptors: 57
Donors: 78
RM index: 8.399760094524142 

RP index: 4.787491742782046 

Sample CR18_XF180416_S has : 4922 kos
Acceptors: 46
Donors: 76
RM index: 8.159374736775426 

RP index: 4.718498871295094 



In [40]:
print(dictio)
#INtersection
common_A = set(all_A[0]).intersection(*all_A[1:])
common_D = set(all_D[0]).intersection(*all_D[1:])
print(common_A)
print(common_D)
# ALL
all_A = set().union(*all_A)
all_D = set().union(*all_D)
print(all_A)
print(all_D)

{'CR18_ER180415_F': {'A': {'K02567', 'K00362', 'K04015', 'K00376', 'K07306', 'K00411', 'K00395', 'K02257', 'K00368', 'K02299', 'K10944', 'K00425', 'K12527', 'K12528', 'K00412', 'K11181', 'K02274', 'K00374', 'K14029', 'K02568', 'K10946', 'K00158', 'K00404', 'K00537', 'K00405', 'K00410', 'K15864', 'K00467', 'K15862', 'K02827', 'K00198', 'K00406', 'K00426', 'K00363', 'K00413', 'K04561', 'K02275', 'K02276', 'K14028', 'K00194', 'K00370', 'K23995', 'K03741', 'K00394', 'K02297', 'K00407', 'K17877', 'K00371', 'K11180', 'K15408', 'K03385', 'K02298'}, 'D': {'K01678', 'K00443', 'K06282', 'K08355', 'K00441', 'K00156', 'K17226', 'K17993', 'K17994', 'K17227', 'K03389', 'K00245', 'K14086', 'K22516', 'K00240', 'K04072', 'K00533', 'K17223', 'K00127', 'K00164', 'K00132', 'K00158', 'K22515', 'K08264', 'K00200', 'K18007', 'K00124', 'K00024', 'K00030', 'K01677', 'K17222', 'K00126', 'K14127', 'K00201', 'K17224', 'K08356', 'K00122', 'K00134', 'K03388', 'K00235', 'K10946', 'K03390', 'K21308', 'K10535', 'K0003

In [42]:
for item in common_A:
    print(item)



K02567
K00362
K00376
K07306
K00411
K00395
K02257
K00368
K00425
K11181
K00412
K02274
K00404
K00405
K00410
K15862
K02827
K00198
K00426
K02275
K02276
K00370
K23995
K03741
K00371
K11180
K15408
K03385


In [ ]:
#ACCEPTORS
dsr = ["K11180","K11181","K27187"] #dsrA,B,gamma
DMSORed = ['K00184', 'K00185', 'K07306', 'K07307', 'K07308', 'K16964']
#DONors
hyd = ['K17993', 'K17994', 'K17995', 'K17996'] #Sulfhydrogenase, (sulfide)n -> (sulfide)n-1 #NOT fo


In [ ]:

#item = "K11180"
for item in hyd:
    if item in all_D:
        print(item)